# 예제 2.1 토큰화 코드

In [2]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print(f"input_text_list : {input_text_list}")

input_text_list : ['나는', '최근', '파리', '여행을', '다녀왔다']


In [3]:
# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word : idx for idx, word in enumerate(input_text_list)}
idx2str = {idx : word for idx, word in enumerate(input_text_list)}
print(f"str2idx : {str2idx}")
print(f"idx2str : {idx2str}")

str2idx : {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str : {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}


In [4]:
# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print(f"input_ids : {input_ids}")

input_ids : [0, 1, 2, 3, 4]


# 예제 2.2 토큰 아이디에서 벡터로 변환

In [5]:
import torch.nn as nn
import torch

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

In [6]:
input_embeddings = embed_layer(torch.tensor(input_ids))
input_embeddings = input_embeddings.unsqueeze(0)
input_embeddings.shape

torch.Size([1, 5, 16])

# 예제 2.3 절대적 위치 인코딩

In [7]:
import torch.nn as nn
import torch

embedding_dim = 16
max_position = 12
position_embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids)) # (5,16)
token_embeddings = token_embeddings.unsqueeze(0) # (1,5,16)
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

# 2.4 쿼리, 키, 값 벡터를 만드는 nn.linear 층

In [8]:
import torch.nn as nn

head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
# (1,5,16)
querys = weight_q(input_embeddings)
print(f"querys : {querys}")

# (1,5,16)
keys = weight_k(input_embeddings)
print(f"keys : {keys}")

# (1,5,16)
values = weight_v(input_embeddings)
print(f"values : {values}")

querys : tensor([[[-1.7437, -0.3612,  1.1627,  0.0903,  1.5031,  0.4264, -0.7657,
          -0.4169, -0.5126, -0.8615,  0.0235, -1.0073,  0.3954, -0.8625,
           0.6526,  0.9804],
         [ 0.4367,  0.6721,  0.6141,  2.1430,  0.7625,  1.7115, -0.4548,
           2.6210, -0.4420,  0.5068, -1.4000,  2.2586, -0.4002,  1.2673,
          -0.1687,  0.6671],
         [ 0.9893,  0.7571, -0.9151,  0.2145, -0.9631, -0.9537, -0.1105,
          -0.6878, -0.1830,  1.0054, -0.1079,  1.2545,  0.5934, -0.6866,
           0.1977, -0.5988],
         [ 1.1696, -0.1538,  1.5511, -1.3463,  0.2340,  0.3890,  0.9434,
          -1.0653,  1.2194, -0.0922,  1.7279, -0.0535,  1.0757,  0.5089,
           0.2536,  0.4214],
         [-0.4564, -0.7045, -0.7121,  1.1968, -1.0562, -1.0733,  0.8456,
           0.5988, -1.3383,  0.6284, -0.4088, -0.2570,  0.0872, -0.0624,
          -0.3653,  0.3548]]], grad_fn=<ViewBackward0>)
keys : tensor([[[-0.7745, -1.2311,  1.0522,  1.7446,  0.7581,  0.7985,  0.3313,
         

# 2.5 스케일 점곱 방식의 어텐션

In [9]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal = False):
    dim_k = querys.size(-1) # 16
    # 1. 쿼리와 키를 곱한다. 분산이 커지는 것을 방지하기 위해 임베딩 차원 수의 제곱근으로 나눈다
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    
    # 2. 쿼리와 키를 곱해 게산한 스코어(scores)를 합이 1이 되도록 소프트맥스를 취해 가중치로 바꾼다
    weights = F.softmax(scores, dim=1)
    
    # 3. 가중치와 값을 곱해 입력과 동일한 형태의 출력을 반환한다
    return weights @ values

# 2.6 어텐션 연산의 입력과 출력

In [10]:
print(f"원본 입력 형태 : {input_embeddings.shape}")

after_attention_embeddings = compute_attention(querys, keys, values)

print(f"어텐션 적용 후 형태 : {after_attention_embeddings.shape}")

원본 입력 형태 : torch.Size([1, 5, 16])
어텐션 적용 후 형태 : torch.Size([1, 5, 16])


# 2.7 어텐션 연산의 입력과 출력

In [11]:
class AttentionHead(nn.Module):
    def __init__(self, token_embed_dim, head_dim, is_causal=False):
        super().__init__()
        self.is_causal = is_causal
        # 쿼리 벡터 생성을 위한 선형 층
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        
        # 키 벡터 생성을 위한 선형 층
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        
        # 값 벡터 생성을 위한 선형 층
        self.weight_v = nn.Linear(token_embed_dim, head_dim)
        
    def forward(self, querys, keys, values):
        outputs = compute_attention(
            # 쿼리 벡터
            self.weight_q(querys),
            
            # 키 벡터
            self.weight_k(keys),
            
            # 값 벡터
            self.weight_v(values),
            is_causal = self.is_causal
        )
    
        return outputs

# 2.8 멀티 헤드 어텐션 구현

In [12]:
class MultiheadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
        super().__init__()
        self.n_head = n_head
        self.is_causal = is_causal
        # 쿼리 벡터 생성을 위한 선형 층
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        
        # 키 벡터 생성을 위한 선형 층
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        
        # 값 벡터 생성을 위한 선형 층
        self.weight_v = nn.Linear(token_embed_dim, head_dim)
        
        self.concat_linear = nn.Linear(d_model, d_model)
        
    def forward(self, querys, keys, values):
        B, T, C = querys.size()
        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1,2)
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1,2)
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1,2)
        attention = compute_attention(querys, keys, values, self.is_causal)
        output = attention.transpose(1,2).contiguous().view(B, T, C)
        output = self.concat_linear(output)
        return output
        
n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

# 2.9 총 정규화

In [15]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
print(norm_x.shape)

# 실제로 평균과 표준편차 확인하기
norm_x.mean(dim=1).data, norm_x.std(dim=1).data

torch.Size([1, 5, 16])


(tensor([[ 0.2042,  0.2211, -0.1273,  0.4861, -0.2177, -0.1498, -0.2115,  0.3610,
          -0.1572,  0.1849, -0.3571, -0.0033,  0.1521, -0.3891,  0.0834, -0.0796]]),
 tensor([[0.9781, 1.1271, 1.0244, 1.4549, 1.0181, 0.8899, 0.9389, 1.1653, 0.7113,
          0.9844, 1.1472, 0.9055, 0.7757, 1.3902, 1.2438, 1.2820]]))

# 2.10 피드 포워드 층 코드드

In [ ]:
class PreLayerNormFeedForward(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)()